In [86]:
import pandas as pd
import numpy as np

In [133]:
DF_COMPRAS_2024 = pd.read_csv(
    "Compras 2024 - Controles Gerais - Lançamento de NF Não Recorrentes.csv",
    usecols=[
        "Marca",
        "UNIDADE",
        "Valor",
        "Fluig",
        "Mês",
    ]
)

DF_DE_PARA_OFC = pd.read_excel(
    'https://docs.google.com/spreadsheets/d/1FXJHfBeenCm7kDq-3SpJjuuBY0nXZ2xF-6DTK2HQyaQ/export?=download', 
    sheet_name="Unidades",
    usecols=[
        "Chave 2024",
        "Marca 2024",
        "Unidade 2024"
    ]
)

In [134]:
DF_COMPRAS_2024_COPY = DF_COMPRAS_2024.copy()
DF_DE_PARA_OFC_COPY = DF_DE_PARA_OFC.copy()

In [135]:
DF_COMPRAS_2024_COPY["Valor"] = DF_COMPRAS_2024_COPY["Valor"] \
                                        .str.replace("R\$", "", regex=True) \
                                        .str.replace(".", "") \
                                        .str.replace(",", ".") \
                                        .str.strip().astype("float")

<>:2: SyntaxWarning: invalid escape sequence '\$'
<>:2: SyntaxWarning: invalid escape sequence '\$'
C:\Users\nando\AppData\Local\Temp\ipykernel_15464\3437852144.py:2: SyntaxWarning: invalid escape sequence '\$'
  .str.replace("R\$", "", regex=True) \


In [136]:
DF_COMPRAS_2024_COPY = DF_COMPRAS_2024_COPY[DF_COMPRAS_2024_COPY["Marca"].notna()]
DF_COMPRAS_2024_COPY = DF_COMPRAS_2024_COPY[DF_COMPRAS_2024_COPY["UNIDADE"].notna()]

In [137]:
DF_COMPRAS_2024_COPY["Unidade"] = DF_COMPRAS_2024_COPY.apply(lambda row: row["UNIDADE"].replace(f"{row['Marca']} ", ""), axis=1)
DF_COMPRAS_2024_COPY["Chave 2024"] = DF_COMPRAS_2024_COPY["Marca"] + DF_COMPRAS_2024_COPY["Unidade"] 

In [138]:
DF_COMPRAS_2024_DE_PARA = pd.merge(DF_COMPRAS_2024_COPY, DF_DE_PARA_OFC_COPY, on="Chave 2024", how="left")

In [139]:
BASE_ERROS = DF_COMPRAS_2024_DE_PARA[DF_COMPRAS_2024_DE_PARA["Marca 2024"].isna()]
DF_COMPRAS_2024_DE_PARA = DF_COMPRAS_2024_DE_PARA[DF_COMPRAS_2024_DE_PARA["Marca 2024"].notna()]

In [140]:
DF_COMPRAS_2024_DE_PARA = DF_COMPRAS_2024_DE_PARA[[
    "Marca 2024",
    "Unidade 2024",
    "Valor",
    "Mês",
]]

DF_COMPRAS_2024_DE_PARA["Ano"] = 2024
DF_COMPRAS_2024_DE_PARA["NO"] = "Ativos"
DF_COMPRAS_2024_DE_PARA["Categoria"] = "Supply"
DF_COMPRAS_2024_DE_PARA["Tipo Valor"] = "Capex"
DF_COMPRAS_2024_DE_PARA["Tipo"] = "Real"

In [141]:
from datetime import datetime

DF_COMPRAS_2024_DE_PARA.loc[DF_COMPRAS_2024_DE_PARA["Valor"].isna(), "Valor"] = 0.0
DF_COMPRAS_2024_DE_PARA.loc[DF_COMPRAS_2024_DE_PARA["Mês"].isna(), "Mês"] = datetime.now().month

In [148]:
DF_COMPRAS_2024_DE_PARA = DF_COMPRAS_2024_DE_PARA[[
    "Valor",
    "Marca 2024",
    "Unidade 2024",
    "Ano",
    "NO",
    "Categoria",
    "Tipo Valor",
    "Tipo",
    "Mês",
]]
DF_COMPRAS_2024_DE_PARA.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3418 entries, 0 to 3649
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Valor         3418 non-null   float64
 1   Marca 2024    3418 non-null   object 
 2   Unidade 2024  3418 non-null   object 
 3   Ano           3418 non-null   int64  
 4   NO            3418 non-null   object 
 5   Categoria     3418 non-null   object 
 6   Tipo Valor    3418 non-null   object 
 7   Tipo          3418 non-null   object 
 8   Mês           3418 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 267.0+ KB


In [146]:
ATIVOS_ATE_ABRIL = pd.read_excel("Ativos até Abril.xlsx")
CONCAT_ATIVOS = pd.concat([ATIVOS_ATE_ABRIL, DF_COMPRAS_2024_DE_PARA], axis=0)

In [151]:
BASE_ERROS = pd.concat([BASE_ERROS, CONCAT_ATIVOS.loc[CONCAT_ATIVOS["Marca 2024"].isna()]], axis=0)
CONCAT_ATIVOS = CONCAT_ATIVOS.loc[CONCAT_ATIVOS["Marca 2024"].notna()]

In [152]:
CONCAT_ATIVOS.to_excel("Ativos 2024 (Padronizado).xlsx", index=False)
BASE_ERROS.to_excel("Dados a Tratar.xlsx", index=False)